In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from analysts_tools.redash_methods import *

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter

In [2]:
df_eagle = get_fresh_query_result("https://internal-redash.federate.frubana.com/",177832,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [3]:
df_offers = get_fresh_query_result("https://internal-redash.federate.frubana.com/",177833,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [4]:
df_bench = get_fresh_query_result("https://internal-redash.federate.frubana.com/",179153,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [5]:
query = """
SELECT DISTINCT 
  --qp.id,
  --qp.name,
  p.region_code,
  --p.name,
  p.name AS sku_name,
  bs2.name AS card_name,
  bs.addl_product_id AS card_id,
  p.product_id AS product_id,
  bs.sku_id,
  p.sku,
  bcat2.name as cat,
  bcat.name as subcat
FROM postgres_main_co."purchase_orders.products"                        p
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"              bs      ON bs.upc = p.sku
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_product"           bp      ON bp.product_id = bs.addl_product_id --Conectar sku con la tarjeta
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_sku"               bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"     bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL --Relaciones categorias 
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"          bcat    ON bp.default_category_id = bcat.category_id --Nombre subcategoria
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"          bcat2   ON bcx.category_id = bcat2.category_id --Nombre categoria

where
  p.deleted_at is null
  and p.parent_id is null
  and bs.archived = 'N'
  and bs.sndbx_id is null
  and bp.archived = 'N'
  and bs.catalog_disc < 0
  and ((bp.archived = 'N' and bp.sndbx_id is null) or (bp.archived is null))
  """
df_aux1 = read_connection_data_warehouse.runQuery(query)

In [6]:
query = """
SELECT  close_date
,       region_code
,       product_id
,CASE
    WHEN region_code IN ('SPO','BHZ','CWB','VCP') THEN SUM(stockout_correction * avg_price_without_discount)/4.75
    WHEN region_code IN ('BOG','BAQ','MDE') THEN SUM(stockout_correction * avg_price_without_discount)/3776
    WHEN region_code IN ('CMX') THEN SUM(stockout_correction * avg_price_without_discount)/19.65
    ELSE SUM(stockout_correction * avg_price_without_discount)
END AS gmv_lost
FROM    lnd_ops.dp_input_historical_product_information
WHERE   close_date >= CURRENT_DATE - 60
        AND warehouse <> 'ALL'
        --AND region_code = 'BOG'
GROUP BY 1, 2, 3
HAVING  SUM(stockout_correction * avg_price_without_discount) > 0
  """
df_so = read_connection_data_warehouse.run_read_prod_query(query)

In [8]:
query =  """
SELECT DISTINCT
    s.identifier_value AS region_code,
    DATE(fs.order_submitted_date) AS submit_date,
    cat.parent_description as category,
    cat.description AS subcategory,
    dp.card_id,
    dp.card_description AS product_name,
    AVG(fs.product_price + (fs.product_tax/fs.product_quantity_x_step_unit)) AS product_price,
    AVG(product_price_discount + (fs.product_tax/fs.product_quantity_x_step_unit)) AS product_price_discount,
    AVG(pb.price) AS benchmark
    
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id  
INNER JOIN dpr_shared.dim_stock_unit         su  ON su.card_id = dp.card_id
INNER JOIN dpr_pricing.snap_daily_benchmark_competitor pb ON pb.dim_stock_unit = su.stock_unit_id AND fs.dim_submitted_date = pb.dim_benchmark_day

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND fs.product_quantity_x_step_unit > 0
    AND DATE(fs.order_submitted_date) >= DATE_TRUNC('day', date(getdate()) - interval '60 day')
GROUP BY 1,2,3,4,5,6
"""
data_ventas_aux = run_read_dwd_query(query)
data_ventas_aux[["product_price", "product_price_discount"]] = data_ventas_aux[["product_price", "product_price_discount"]].astype(float)
data_ventas_aux['submit_date'] = pd.to_datetime(data_ventas_aux['submit_date']) 

In [ ]:
df_so = pd.merge(df_so, df_aux1[['region_code','product_id','card_id']],  how='left', left_on=['region_code','product_id'], right_on = ['region_code','product_id'])

In [ ]:
df = pd.merge(df_eagle, df_aux1[['region_code','product_id','card_id','sku_name','cat','subcat']],  how='left', left_on=['region_code','product_id'], right_on = ['region_code','product_id'])

In [ ]:
df = df.dropna()

In [ ]:
# Assuming df is your DataFrame
def calculate_fit_for_discount(row):
    if row['cat'] in ['Frutas e Verduras', 'Frutas & Verduras']:
        if row['clearence_days'] < 1:
            return 'FALSE'
        elif 1 <= row['clearence_days'] < 2:
            return 'check_with_supply'
        else:
            return 'TRUE'
    elif row['cat'] in ['Abarrotes', 'Abarrotes & Despensa', 'Mercearia', 'Bebidas', 'Congelados', 'Despensa']:
        if row['clearence_days'] <= 3:
            return 'FALSE'
        elif 3 < row['clearence_days'] <= 5:
            return 'check_with_supply'
        else:
            return 'TRUE'
    elif row['cat'] in ['Lácteos & Huevos', 'Lácteos Y Huevos', 'Laticínios e Ovos', 'Carne, Pollo & Pescados', 'Carnes, Aves e Peixes']:
        if row['clearence_days'] <= 1:
            return 'FALSE'
        elif 1 < row['clearence_days'] <= 3:
            return 'check_with_supply'
        else:
            return 'TRUE'
    elif row['cat'] in ['Aseo e Higiene', 'Desechables', 'Descartáveis', 'Limpeza e Higiene']:
        if row['clearence_days'] <= 2:
            return 'FALSE'
        elif 2 < row['clearence_days'] <= 4:
            return 'check_with_supply'
        else:
            return 'TRUE'
    else:
        return 'FALSE'

df['fit_for_discount'] = df.apply(calculate_fit_for_discount, axis=1)

In [ ]:
# Convert date columns to a common format
df_so['close_date'] = pd.to_datetime(df_so['close_date'])
df['operation_date'] = pd.to_datetime(df['operation_date'])
df_bench['fecha'] = pd.to_datetime(df_bench['fecha'])
df_offers['created_at'] = pd.to_datetime(df_offers['created_at'])

In [ ]:
df3 = pd.merge(df_offers, df[['region_code','operation_date','card_id','clearence_days','fit_for_discount']],  how='left', left_on=['city','created_at','card_id'], right_on = ['region_code','operation_date','card_id'])

In [ ]:
df3 = df3.dropna().reset_index(drop=True)

In [ ]:
import pandas as pd
from datetime import timedelta

# Assuming df3 is your offers DataFrame and df_so is your stock DataFrame

# Convert date columns to datetime format
df3['created_at'] = pd.to_datetime(df3['created_at'])
df_so['close_date'] = pd.to_datetime(df_so['close_date'])

# Perform the left join
merged_df = df3.merge(df_so[['card_id', 'close_date', 'gmv_lost']], how='left', on='card_id')

# Add condition to create null values for rows that don't match the condition
merged_df.loc[
    ~(
        (merged_df['close_date'] >= merged_df['created_at']) &
        (merged_df['close_date'] <= merged_df['created_at'] + timedelta(days=5))
    ),
    ['close_date', 'gmv_lost']
] = [None, None]


merged_df['close_date'] = merged_df['close_date'].fillna(merged_df['created_at'])
merged_df['gmv_lost'] = merged_df['gmv_lost'].fillna(0)
#Filter rows based on conditions
merged_df = merged_df[
    ((merged_df['close_date'] >= merged_df['created_at']) &
     (merged_df['close_date'] <= merged_df['created_at'] + timedelta(days=5)))
]

In [ ]:
merged_df = merged_df.drop_duplicates()

In [ ]:
# Custom function to count non-zero and non-null values
def custom_count(series):
    return len([x for x in series if pd.notna(x) and x != 0])

# Assuming 'merged_df' is your DataFrame
result_df = merged_df.groupby(by=['city', 'responsable', 'created_at',
       'card_id', 'name', 'cat', 'subcat', 'region_code', 'operation_date',
       'clearence_days', 'fit_for_discount']).agg({'gmv_lost': [np.sum, custom_count]}).reset_index()

# Rename columns for clarity
result_df.columns = ['city', 'responsable', 'created_at',
       'card_id', 'name', 'cat', 'subcat', 'region_code', 'operation_date',
       'clearence_days', 'fit_for_discount', 'total_gmv_lost', 'count_non_zero']

In [ ]:
data_ventas_aux["dct"] = 1-(data_ventas_aux.product_price_discount/data_ventas_aux.product_price)

In [ ]:
df2 = pd.merge(data_ventas_aux, df_bench,  how='left', left_on=['region_code','submit_date','card_id'], right_on = ['region_code','fecha','card_id'])
df2['bench'] = np.where(df2['benchmark_y'].notnull(), df2['benchmark_y'], df2['benchmark_x'])
df2['gpi'] = df2.product_price/df2.bench
df2['npi'] = df2.product_price_discount/df2.bench
columns_to_drop = ['benchmark_x', 'benchmark_y', 'fecha']
df2.drop(columns=columns_to_drop, inplace=True)

In [ ]:
#df2 = df2.loc[(df2.gpi>0.6) & (df2.gpi<2)].sort_values("gpi").reset_index(drop=True)

In [31]:
# Assuming your DataFrame is named 'df2'
# Sort the DataFrame by 'card_id' and 'submit_date' in ascending order
df2.sort_values(by=['card_id', 'submit_date'], inplace=True)

# Calculate the percentage change for 'gpi', 'dct', and 'npi'
df2['gpi_percent_change'] = df2.groupby('card_id')['gpi'].pct_change() #* 100
df2['dct_percent_change'] = df2.groupby('card_id')['dct'].pct_change() #* 100
df2['npi_percent_change'] = df2.groupby('card_id')['npi'].pct_change() #* 100

# Drop the first row for each 'card_id' since percentage change is not available
df2 = df2.dropna()

# Reset the index if needed
df2.reset_index(drop=True, inplace=True)

In [32]:
# Calculate the rolling average for each card_id and submit_date
rolling_period = 3  # Adjust this value as needed

for col in ['gpi_percent_change', 'dct_percent_change', 'npi_percent_change']:
    df2[f'{col}_avg_t-3'] = df2.groupby('card_id')[col].transform(lambda x: x.rolling(rolling_period).mean())
    df2[f'{col}_avg_t+3'] = df2.groupby('card_id')[col].transform(lambda x: x.shift(-2).rolling(rolling_period).mean())
    
df2 = df2.dropna()

In [38]:
df2.to_excel("C2.xlsx")

In [34]:
df_final = pd.merge(result_df, df2[['region_code','submit_date','card_id','gpi_percent_change_avg_t-3',
       'gpi_percent_change_avg_t+3', 'dct_percent_change_avg_t-3',
       'dct_percent_change_avg_t+3', 'npi_percent_change_avg_t-3',
       'npi_percent_change_avg_t+3']],  how='left', left_on=['city','created_at','card_id'], right_on = ['region_code','submit_date','card_id'])

In [35]:
df_final = df_final.dropna()

In [36]:
df_final.to_excel("B2.xlsx")

In [37]:
result_df.to_excel("A2.xlsx")